In [1]:
### Step 1: import modules

In [ ]:
import tensorflow as tf
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
import tqdm
from sklearn.preprocessing import LabelBinarizer

In [ ]:
### Step 2: setup variables

In [ ]:
BASE_PATH = '/Users/subhi/Downloads/demo/Data1'
VIDEOS_PATH = os.path.join(BASE_PATH, '**','*.mp4')

#this specifies the sequence length will process by LSTM
SEQUENCE_LENGTH = 40
#this specifies the Batch size that will be processed by LSTM
BATCH_SIZE = 16
print(VIDEOS_PATH)

In [ ]:
###STEP 3: Create a MyLabelBinarizer for 2 Classes

In [ ]:
class MyLabelBinarizer(LabelBinarizer):
    def transform(self, y):
        Y = super().transform(y)
        if self.y_type_ == 'binary':
            return np.hstack((Y, 1-Y))
        else:
            return Y
    def inverse_transform(self, Y, threshold=None):
        if self.y_type_ == 'binary':
            return super().inverse_transform(Y[:, 0], threshold)
        else:
            return super().inverse_transform(Y, threshold)

In [ ]:
LABELS = ['Stairs','Doors']
encoder = MyLabelBinarizer()
encoder.fit(LABELS)

In [ ]:
###STEP 4:

###Create the LSTM model:    
### 1) Masking layer  2) LSTM layer with 512 cells, dropout 0.5, recurrent_dropout of 0.5  

###3) a fully connected relu activation layer with 256 outputs, 4) a droupout layer 0.5
### 5) a final decision fully connected layer of putput length of labels (which is the number of classes) 
### with softmax activation.
model = tf.keras.Sequential([
    tf.keras.layers.Masking(mask_value=0.),
    tf.keras.layers.LSTM(512, dropout=0.5, recurrent_dropout=0.5),
    tf.keras.layers.Dense(256, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(2, activation='softmax')
    #tf.keras.layers.Dense(len(LABELS), activation='softmax')
])

In [ ]:
### Step 5: Setup for the model the Loss function, the Optimizer function, and any metrics want to compute in training

In [ ]:
model.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy', 'top_k_categorical_accuracy'])

In [ ]:
### STEP 6:

## Setup the training and test list which are lists of the training filenames.  
## Defined a function make_generator that returns a generator which will randomly shuffle 
## a file list (either training or testing that will be passed later) and 
## then changes the file extension of the mp4 files listed in the list to .npy which is our features
## for that mp4 video which were calcluated in featureExtractionNotebook last step


In [ ]:
train_file = '/Users/subhi/Downloads/demo/Data1/trainlist.txt'
test_file = '/Users/subhi/Downloads/demo/Data1/testlist.txt'
#test_file = 'C:/Grewe/Classes/CS663/Mat/LSTM/data/testlist01.txt'
#train_file = 'C:/Grewe/Classes/CS663/Mat/LSTM/data/trainlist01.txt'

with open(test_file) as f:
    test_list = [row.strip() for row in list(f)]

with open(train_file) as f:
     train_list = [row.strip() for row in list(f)]
     #train_list=[row.split(' ')[0] for row in train_list]
     train_list = [row.split(' ')[0] for row in train_list]
#print(train_list)
def make_generator(file_list):
    def generator():
        np.random.shuffle(file_list)
        for path in file_list:
            full_path = os.path.join(BASE_PATH, path).replace('.mp4', '.npy')
            
            label = os.path.basename(os.path.dirname(path))
            features = np.load(full_path)
            
            
            padded_sequence = np.zeros((SEQUENCE_LENGTH, 1280))
            padded_sequence[0:len(features)] = np.array(features)
            
            transformed_label = encoder.transform([label])
            
            yield padded_sequence, transformed_label[0]
    return generator

In [ ]:
print(train_list)

In [ ]:
print(test_list)

In [ ]:
### STEP 7:

### Setup the train_dataset and valid_dataset (validation/testing). 
### Here we are setting up training batch sets of 16, as defined above.
#for tesnorflow 2.*
train_dataset = tf.data.Dataset.from_generator(make_generator(train_list),
                output_types=(tf.float32, tf.int16),
                output_shapes=((SEQUENCE_LENGTH, 1280), (len(LABELS))))
                 

train_dataset = train_dataset.batch(BATCH_SIZE,drop_remainder=True).prefetch(tf.data.experimental.AUTOTUNE)


valid_dataset = tf.data.Dataset.from_generator(make_generator(test_list),
                 output_types=(tf.float32, tf.int16),
                 output_shapes=((SEQUENCE_LENGTH, 1280), (len(LABELS))))
valid_dataset = valid_dataset.batch(BATCH_SIZE,drop_remainder=True).prefetch(tf.data.experimental.AUTOTUNE)


In [ ]:
print(train_dataset)

In [ ]:
print(valid_dataset)

In [ ]:
BASE_DATA_PATH = '/Users/subhi/Downloads/demo/Data1'
mylog_dir = os.path.join( BASE_DATA_PATH, "train_log")
print("Mylog directory = " + mylog_dir)

In [ ]:
tensorboard_callback = tf.keras.callbacks.TensorBoard(os.path.join('tmp'), update_freq=1000)
model.fit(train_dataset, epochs=50, callbacks=[tensorboard_callback], validation_data=valid_dataset)

In [ ]:
model.summary()

In [ ]:
### STEP 8:  save the tensorflow model to an h5 file

In [ ]:
model.file=os.path.join(BASE_PATH,'my_model.h5')
model.save(model.file)

In [ ]:
### STEP 9: Run predictions on the test data feature extracted

In [ ]:
# make predictions

# Generate predictions (probabilities -- the output of the last layer)
# on new data using `predict`
print('\n# Generate predictions ')
predictions = model.predict(valid_dataset, verbose=1 )

In [ ]:
#print out prediction info for validation data set (as do not have separate test data set)
print('predictions shape:', predictions.shape)
print(predictions)
print(len(predictions))

In [ ]:
# ## OPTIONAL:  Save to a SavedModel

#create directory to save the SavedModel
#setup directories
saved_model_dir1 = os.path.join(BASE_DATA_PATH, 'saved_model')
print(" path exists=" + str(os.path.exists(saved_model_dir1)))
if(os.path.exists(saved_model_dir1) == False):
    print(" creating" + str(saved_model_dir1))
    os.mkdir(saved_model_dir1)
    
saved_model_dir = os.path.join(BASE_DATA_PATH, 'saved_model\LSTM_SavedModel')    
if(os.path.exists(saved_model_dir) == False):
    print(" creating" + str(saved_model_dir))
    os.mkdir(saved_model_dir)

In [ ]:
#save model as SavedModel
tf.keras.experimental.export_saved_model(model, saved_model_dir)

In [ ]:
%load_ext tensorboard
%tensorboard --logdir="train_log" --host localhost --port=8084

In [ ]:
%reload_ext tensorboard

In [ ]:
print("     --shape is" + str(predictions.shape))